## Task 2 - Classification/Regression (Use all features)

Write a Python program to build a MLP classifier and evaluate your model’s performance. For this task, you should use the pre-processed and transformed dataset without feature selection from Task 3.1.

<span style="font-weight:bold;">Save your response (program and markdown) as task2.ipynb</span>

In [1]:
# when to use linear regression

# TRY GPU FOR RUNNING PROGRAMS

In [2]:
import numpy as np
from sklearn.svm import SVC
import matplotlib.pyplot as plt 
from functions.pre_processing import *
from functions.transformations import *
from functions.feature_selection import *
from functions.analysis import *
from functions.classifiers import *
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

-----
## Pre-Processing and Transformation

In [3]:
x_train, x_test, y_train, y_test = fetch_datasets()
x_train_norm, x_test_norm = normalise_min_max(x_train, x_test)
# x_train_norm, x_test_norm = normalise_std_scaler(x_train, x_test)

# x_train_norm, x_test_norm = normalise_robust_scaler(x_train, x_test)



---
## MLP Classifier Training

In [4]:
mlp_training = mlp_classifier(x_train, x_test, y_train, y_test, "holdout")
mlp_training = mlp_classifier(x_train, x_test, y_train, y_test, "cv")
mlp_training = mlp_classifier(x_train, x_test, y_train, y_test, "k_fold")
mlp_training = mlp_classifier(x_train, x_test, y_train, y_test, "k_fold_strat")


------ Holdout Validation ------
Training Accuracy: 1.0
Testing Accuracy: 0.9255555555555556


---
## Hyperparameter Optimisation

In [ ]:
# Use nested CV here
# Confusion Matrix & Classification Report

# Make this a module?

# max_iters?
hidden_layers = [(50,50,50), (50,100,50), (100,)]
activations = ['tanh', 'relu']
solvers = ['sgd', 'adam']
alphas = [0.0001, 0.05]

outer_cv = KFold(n_splits=10, shuffle=True)

for layer in range(len(hidden_layers)):
     for activation in range(len(activations)):
          for solver in range(len(solvers)):
               for alpha in alphas:
                    inner_scores = []
                    mlp = MLPClassifier(hidden_layer_sizes=hidden_layers[layer], activation=activations[activation], solver=solvers[solver], alpha=alphas[alpha])
                    mlp.fit(x_train, y_train)
                    current_train = mlp.score(x_train, y_train) 
                    current_test = mlp.score(x_test, y_test) 

                    for train_index, val_index in outer_cv.split(x_train):
                         X_train, X_val = x_train[train_index], x_train[val_index]
                         Y_train, Y_val = y_train[train_index], y_train[val_index]
                         mlp_classifier.fit(X_train, Y_train)
                         training_score = mlp_classifier.score(X_train, Y_train)
                         test_score = mlp_classifier.score(X_val, Y_val)
                         inner_scores.append(test_score)
                    
                    mean_score = sum(inner_scores) / len(inner_scores)

                    print(f"------- With hidden_layers={hidden_layers[layer]}, activation={activations[activation]}, solver={solvers[solver]}, alpha={alphas[alpha]}")
                    print(f"Mean - {mean_score}")

---
## Evaluation